In [0]:
QUERYNAME = 'consulta_bike_sales'
CATALOGNAME= 'c_bikeshop' 
SCHEMANAME = 'gold'
TABLENAME = 'bike_sales'

In [0]:
spark.sql("USE CATALOG c_bikeshop")


In [0]:
def import_query(path):
    with open(path, 'r') as open_file:
        return open_file.read()

In [0]:
path = f'/Workspace/Users/databrickstreinamento@gmail.com/bikestore/SQL/{QUERYNAME}.sql'
source_query = import_query(path)

spark.sql(
    f"""
    CREATE TABLE IF NOT EXISTS {CATALOGNAME}.{SCHEMANAME}.{TABLENAME} 
    AS
    SELECT * FROM (
        {source_query}
    ) LIMIT 0
    """
)

In [0]:
is_empty = spark.sql(f"SELECT COUNT(*) FROM {CATALOGNAME}.{SCHEMANAME}.{TABLENAME}").first()[0] == 0

if is_empty:
    print(f"Tabela {TABLENAME} está vazia. Executando carga full.")
    spark.sql(
        f""" 
        INSERT OVERWRITE {CATALOGNAME}.{SCHEMANAME}.{TABLENAME} 
        SELECT * FROM (
            {source_query}
            )
        """
    )
else:
    print(f"Tabela {TABLENAME} não está vazia. Executando carga incremental.")
    
    merge_sql = f""" 
        MERGE INTO {CATALOGNAME}.{SCHEMANAME}.{TABLENAME} AS target
        USING (
            {source_query}
            ) AS source
        ON target.order_id = source.order_id AND target.item_id = source.item_id
        WHEN MATCHED THEN
            UPDATE SET *
        WHEN NOT MATCHED
            THEN INSERT *
        WHEN NOT MATCHED BY SOURCE
            THEN DELETE
        """
    
    spark.sql(merge_sql)